# Finetuning SDXL using DreamBooth
**In order to use this colab upload images to a folder named `images/`**

*You can upload them while the `pip install` command runs*

In [ ]:
!mkdir images/

In [ ]:
import os
!pip install -U autotrain-advanced > install_logs.txt
!autotrain setup > setup_logs.txt

## Setting up hyperparameters
After your images are uploaded, you can tune the hyperparamenters on the following cell:

1. change model if you wish, you can also select sd2/2.1 or sd1.5
2. Update prompt and remember that you must choose a very strange token for describing yourself (usually "sks" works).


In [ ]:
project_name = 'my_dreambooth_project'
model_name = 'stabilityai/stable-diffusion-xl-base-1.0'
prompt = 'photo of a sks person'

# Hyperparams to play with if you want
learning_rate = 1e-4
num_steps = 1000
batch_size = 1
gradient_accumulation = 4
resolution = 512  # can be 1024, but will be slower
use_8bit_adam = True
use_xformers = True
use_fp16 = True
train_text_encoder = False  # Will be slower, and Colab can close our session
gradient_checkpointing = True

os.environ["PROJECT_NAME"] = project_name
os.environ["MODEL_NAME"] = model_name
os.environ["PROMPT"] = prompt
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_STEPS"] = str(num_steps)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["RESOLUTION"] = str(resolution)
os.environ["USE_8BIT_ADAM"] = str(use_8bit_adam)
os.environ["USE_XFORMERS"] = str(use_xformers)
os.environ["USE_FP16"] = str(use_fp16)
os.environ["TRAIN_TEXT_ENCODER"] = str(train_text_encoder)
os.environ["GRADIENT_CHECKPOINTING"] = str(gradient_checkpointing)

In [ ]:
assert len(os.listdir('images/')) > 0, "You must place your training images inside the images folder!"

## Train!

In [ ]:
!autotrain dreambooth \
--model ${MODEL_NAME} \
--project-name ${PROJECT_NAME} \
--image-path images/ \
--prompt "${PROMPT}" \
--resolution ${RESOLUTION} \
--batch-size ${BATCH_SIZE} \
--num-steps ${NUM_STEPS} \
--checkpointing_steps=500 \
--gradient-accumulation ${GRADIENT_ACCUMULATION} \
--lr ${LEARNING_RATE} \
$( [[ "$USE_FP16" == "True" ]] && echo "--fp16" ) \
$( [[ "$USE_XFORMERS" == "True" ]] && echo "--xformers" ) \
$( [[ "$TRAIN_TEXT_ENCODER" == "True" ]] && echo "--train-text-encoder" ) \
$( [[ "$USE_8BIT_ADAM" == "True" ]] && echo "--use-8bit-adam" ) \
$( [[ "$GRADIENT_CHECKPOINTING" == "True" ]] && echo "--gradient-checkpointing" )

## Inference (generate your images)

In [ ]:
!mkdir $PROJECT_NAME

In [ ]:
!mkdir generations

*You must upload your safetensors file to the project's folder in case you're making inference on a freshly created session (note they're destroyed once a session finishes!)*

In [ ]:
if len([f for f in os.listdir(os.environ['PROJECT_NAME']) if '.safetensors' in f]) < 1:
    raise RuntimeError("You should upload your finetuned weights to the project's folder! (the .safeternsors file)")

In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch

prj_path = "my_dreambooth_project"
model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(
    model,
    torch_dtype=torch.float16,
)
pipe.to("cuda")
pipe.load_lora_weights(prj_path, weight_name="pytorch_lora_weights.safetensors")


# Optionally, you can load the refiner model to improve the quality of our generations
# Beware: Using this on regular Colab will crash it:
#refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
#    "stabilityai/stable-diffusion-xl-refiner-1.0",
#    torch_dtype=torch.float16,
#)
#refiner.to("cuda")

In [ ]:
seed = 42
generator = torch.Generator("cuda")#.manual_seed(seed)

In [ ]:
n_images_generated = 0

In [ ]:
prompt = "a professional LinkedIn close up photography of a sks person with a suit. Good lightning, 4k, good face."
negative_prompt = 'ugly, deform, disfigured, showing teeth, blue eyes, fat, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, extra limbs, disfigured, deformed, body out of frame, bad anatomy, watermark, signature, cut off, low contrast, underexposed, overexposed, bad art, beginner, amateur, distorted face, blurry, draft, grainy'

n_images = 10
n_inference_steps = 100
guidance_scale = 6.5

# We generate and save images in case colab closes our session
for i in range(n_images):
    image = pipe(prompt=prompt,
                generator=generator,
                num_inference_steps=n_inference_steps,
                guidance_scale= guidance_scale,
                negative_prompt=negative_prompt,
                num_images_per_prompt=1) #
    #image = refiner(prompt=prompt, generator=generator, image=image).images[0]
    image.images[0].save(f"generations/generated_image_{n_images_generated}.png")
    n_images_generated += 1

### Optional: Run the refiner model to improve the quality of the generatios

In [ ]:
# Optiona: Run the refiner model on the generated image(s) in case you loaded it!
#seed = 42
#generator = torch.Generator("cuda")
#orig_image = PIL.Image.open('generated_image.png')
#image = refiner(prompt=prompt, generator=generator, image=orig_image, guidance_scale= -1, num_inference_steps=300, negative_prompt='ugly, deform, disfigured, showing teeth').images[0]